In [1]:
from os.path import join, exists, isfile, isdir, abspath, dirname, basename, realpath
from os import makedirs, listdir, pardir, getcwd

import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual, Dropdown, Text, GridBox, VBox, HBox, Accordion, BoundedIntText, Checkbox, Layout, IntProgress, Tab, Image, Button
from IPython.display import display, Javascript

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

ModuleNotFoundError: No module named 'ipywidgets'